# Parallel computing with MPI-3 RMA and Julia

<br/>
### JuliaCon 2018
<br/>
[Bart Janssens](https://github.com/barche)

## Bird's eye view

1. Classic MPI message passing
2. Native Julia parallelism
3. MPI-3 one-sided "Remote Memory Access" (RMA)
4. Applications:
    - MPI Arrays
    - Julia IO on MPI
    - New clustermanager

## The MPI way
* MPI: Message passing interface
* Original: **cooperative** communication

|Rank 0|Rank 1|
|---|---|---|
|`Init()`|`Init()`|
|`mymessage = "Hi 1!"`||
|`Send(mymessage,1)`|`Recv(buffer)`|
|`Finalize()`|`Finalize()`|



The same program is sent to all ranks (run with `mpirun -np 4 sendrecv.jl`)
```julia
using MPI

MPI.Init()

const comm = MPI.COMM_WORLD
const rank = MPI.Comm_rank(comm)
const tag = 0

if rank == 0
  msg = unsafe_wrap(Vector{UInt8},"Hi 1!")
  println("Sending message from rank 0 to rank 1")
  MPI.Send(msg, 1, tag, comm)
elseif rank == 1
  rec_buf = fill(UInt8('a'), 10)
  MPI.Recv!(rec_buf, 0, tag, comm)
  println("Received message $(String(rec_buf)) from rank 0")
end

println("Rank $rank is finalizing.")
MPI.Finalize()
```